<a href="https://colab.research.google.com/github/aldofranco91/Trading/blob/main/Conv_Classification_ResNet_Jaime.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Wed Feb  9 20:02:06 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   62C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install yfinance

In [3]:
import matplotlib
# importing the requests library 
import requests 
import pandas as pd
import numpy as np
import time
# Viz
import matplotlib.pyplot as plt # basic plotting
import seaborn as sns # for prettier plots

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from pickle import dump, load

from IPython import display

import tensorflow as tf

from sklearn import metrics
from scipy import stats

tf.test.gpu_device_name()

'/device:GPU:0'

In [4]:
pd.set_option("display.max_columns", None)

import warnings
warnings.filterwarnings('ignore')

# Get data

In [44]:
folder = '/content/drive/MyDrive/Trading/7_days_class/'
# ajustar valores
past_t = 7
forecast_t = 1

In [45]:
import yfinance as yf
import datetime as dt

start = dt.datetime(2018,2,1)
end = dt.datetime.now() - dt.timedelta(days=1)

df = yf.download('BTC-USD', start, end).drop('Adj Close', axis=1)
df

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Volume
Date,,,,,
2018-02-01,10237.299805,10288.799805,8812.280273,9170.540039,9959400448
2018-02-02,9142.280273,9142.280273,7796.490234,8830.750000,12726899712
2018-02-03,8852.120117,9430.750000,8251.629883,9174.910156,7263790080
2018-02-04,9175.700195,9334.870117,8031.220215,8277.009766,7073549824
2018-02-05,8270.540039,8364.839844,6756.680176,6955.270020,9285289984
...,...,...,...,...,...
2022-02-04,37149.265625,41527.785156,37093.628906,41500.875000,29412210792
2022-02-05,41501.480469,41847.164062,41038.097656,41441.164062,19652846215
2022-02-06,41441.121094,42500.785156,41244.906250,42412.433594,16142097334


# Indicators

In [46]:
# Fear index
df_fear = pd.read_csv('/content/drive/MyDrive/Trading/fear_index.csv')
df_fear = df_fear[df_fear.date.between('2018-02-01', '2022-01-31')]
df_fear['date'] = pd.to_datetime(df_fear['date'])
df['date'] = pd.to_datetime(df.index)
df = df.join(df_fear.set_index('date'), on='date')
df = df.drop(['date'], axis=1)
df['value'] = df['value'].fillna(25) # Fill 3 consequtive days taking sorrounding values (23-26)

In [47]:
price = 'Close'

# MovingAverage
#https://towardsdatascience.com/building-a-comprehensive-set-of-technical-indicators-in-python-for-quantitative-trading-8d98751b5fb
values_ma = range(2,30,2)
for i in values_ma:
    df["maclose" + str(i)] = df[price].rolling(window=i).mean()

# ExponentialMovingWindow
values_ema = range(2,30,2)
for i in values_ema:
    df["ema" + str(i)] = df[price].ewm(span = i, adjust = False).mean()

# Wight Moving Average
# https://predictivehacks.com/?all-tips=how-to-apply-a-rolling-weighted-moving-average-in-pandas
weights = np.array([0.1, 0.2, 0.3, 0.4]) #Tienen que sumar 1 🙂
df['wma'] = df[price].rolling(4).apply(lambda x: np.sum(weights*x))

# RSI
# https://stackoverflow.com/questions/20526414/relative-strength-index-in-python-pandas
values_rsi = range(2,30,2)
close = df[price]

# Get the difference in price from previous step
delta = close.diff()

# Make the positive gains (up) and negative gains (down) Series
up, down = delta.copy(), delta.copy()
up[up < 0] = 0
down[down > 0] = 0

for i in values_rsi:
    # Calculate the EWMA
    roll_up1 = up.ewm(span=i).mean()
    roll_down1 = down.abs().ewm(span=i).mean()

    ## Calculate the RSI based on EWMA
    RS1 = roll_up1 / roll_down1
    RSI1 = 100.0 - (100.0 / (1.0 + RS1))

    # Calculate the SMA
    roll_up2 = up.rolling(i).mean()
    roll_down2 = down.abs().rolling(i).mean()

    # Calculate the RSI based on SMA
    RS2 = roll_up2 / roll_down2
    RSI2 = 100.0 - (100.0 / (1.0 + RS2))

    df["rsiewma" + str(i)] = RSI1.values
    df["rsisma" + str(i)]  = RSI2.values

# BollingerBand
values_bollinger = range(2,30,2)
for i in values_bollinger:
    df['bollingerupper' + str(i)] = df[price].rolling(window=i).mean() + (df[price].rolling(window=i).std() * 2)
    df['bollingerlower' + str(i)] = df[price].rolling(window=i).mean() - (df[price].rolling(window=i).std() * 2)

In [48]:
df['diff'] = df['Close'].shift(-forecast_t) - df['Close'] 
df['percen_diff'] = (df['diff']/df['Close'])*100 

In [49]:
df.percen_diff.describe()

count    1468.000000
mean        0.183195
std         3.869176
min       -37.169539
25%        -1.529662
50%         0.143351
75%         1.796651
max        18.746474
Name: percen_diff, dtype: float64

In [50]:
def get_target(x):
  # reducir margenes
  if x<-1:
    return "sell"
  elif x>1:
    return "buy"
  else:
    return "hold"

df['target'] = df['percen_diff'].apply(lambda x: get_target(x))
df = df.join(pd.get_dummies(df.target))

In [51]:
df = df.dropna()
df.tail(100)

,Open,High,Low,Close,Volume,value,maclose2,maclose4,maclose6,maclose8,maclose10,maclose12,maclose14,maclose16,maclose18,maclose20,maclose22,maclose24,maclose26,maclose28,ema2,ema4,ema6,ema8,ema10,ema12,ema14,ema16,ema18,ema20,ema22,ema24,ema26,ema28,wma,rsiewma2,rsisma2,rsiewma4,rsisma4,rsiewma6,rsisma6,rsiewma8,rsisma8,rsiewma10,rsisma10,rsiewma12,rsisma12,rsiewma14,rsisma14,rsiewma16,rsisma16,rsiewma18,rsisma18,rsiewma20,rsisma20,rsiewma22,rsisma22,rsiewma24,rsisma24,rsiewma26,rsisma26,rsiewma28,rsisma28,bollingerupper2,bollingerlower2,bollingerupper4,bollingerlower4,bollingerupper6,bollingerlower6,bollingerupper8,bollingerlower8,bollingerupper10,bollingerlower10,bollingerupper12,bollingerlower12,bollingerupper14,bollingerlower14,bollingerupper16,bollingerlower16,bollingerupper18,bollingerlower18,bollingerupper20,bollingerlower20,bollingerupper22,bollingerlower22,bollingerupper24,bollingerlower24,bollingerupper26,bollingerlower26,bollingerupper28,bollingerlower28,diff,percen_diff,target,buy,hold,sell
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-10-31,61850.488281,62406.171875,60074.328125,61318.957031,32241199927,74.0,61603.894531,61514.472656,60817.345052,61109.341309,61096.061328,61596.968424,61817.977958,61743.593018,61489.608941,61012.755859,60568.703835,60060.314779,59639.038011,58972.953683,61471.231054,61386.530841,61323.459532,61271.284927,61166.798553,60990.749539,60749.164121,60457.043581,60130.242404,59782.278861,59423.547610,59061.540016,58701.393936,58346.475249,61602.039062,26.649901,0.000000,47.693152,80.470705,49.923215,40.659668,50.529448,49.683208,51.247001,46.817783,52.130428,42.460159,53.063933,49.472079,53.959510,49.417097,54.770702,57.011166,55.479988,56.235654,56.086492,59.435399,56.597705,60.789539,57.024544,62.188720,57.378653,65.067896,62409.819485,60797.969577,62920.984099,60107.961214,63513.326257,58121.363847,63871.906642,58346.775975,63555.383183,58636.739474,65205.914018,57988.022831,65430.775579,58205.180336,65140.001150,58347.184885,65299.021618,57680.196264,65681.335136,56344.176583,65924.432322,55212.975349,66236.337987,53884.291570,66283.198391,52994.877631,67049.304882,50896.602484,-314.550781,-0.512975,hold,0,1,0
2021-11-01,61320.449219,62419.003906,59695.183594,61004.406250,36150572843,74.0,61161.681641,61610.040039,60924.113932,61118.537598,61127.275391,61181.265951,61745.001395,61750.607178,61694.213542,61260.923242,60852.014205,60353.504720,59856.074820,59397.650251,61160.014518,61233.681005,61232.301451,61211.978554,61137.272680,60992.850571,60783.196405,60521.439189,60222.259651,59898.671945,59561.013579,59216.969315,58871.987441,58529.780836,61398.012500,13.564493,0.000000,39.261251,56.755343,45.086107,54.675639,47.229602,50.316144,48.734381,51.219328,50.085169,36.220010,51.327596,47.685071,52.443879,50.241858,53.421911,56.535701,54.262630,58.379863,54.975894,59.227894,55.575853,60.061227,56.077800,57.690882,56.496387,63.848403,61606.523622,60716.839660,62712.206175,60507.873903,63584.381312,58263.846553,63878.875389,58358.199806,63571.697656,58682.853126,63498.875926,58863.655975,65380.892219,58109.110572,65139.969399,58361.244956,64903.881148,58484.545935,65302.283336,57219.563148,65540.550178,56163.478231,65964.578635,54742.430806,66284.102883,53428.046757,66517.171509,52278.128993,2221.996094,3.642353,buy,1,0,0
2021-11-02,60963.253906,64242.792969,60673.054688,63226.402344,37746665647,73.0,62115.404297,61859.649414,61714.783203,61141.859863,61310.553906,61265.951823,61671.030692,61855.156250,61784.905382,61552.188477,61112.996626,60697.595540,60218.398588,59815.921317,62537.606402,62030.769540,61802.044563,61659.628285,61517.114437,61336.473921,61108.957197,60839.670148,60538.485197,60215.598650,59879.743037,59537.723957,59194.536693,58853.685767,62044.557422,92.421634,87.599252,80.289052,64.488777,71.957194,82.985176,66.876168,50.794046,63.981473,56.399238,62.400419,53.072114,61.575650,47.652054,61.174622,53.377797

In [52]:
df_rs = df.drop(['diff', 'percen_diff', 'target'], axis=1)
df_rs = np.array(df_rs)

In [53]:
def split_sequence(sequence, n_steps_in):
    X, y = list(), list()
    range = np.arange(start = 0, stop = len(sequence), step = 1) # Ojo con el step que te fundio :)
    for i in range:
        # find the end of this pattern
        end_ix = i + n_steps_in
        # gather input and output parts of the pattern
        seq_x = sequence[i:end_ix, :-3]
        seq_y = sequence[i:end_ix, -3:][-1]
        if len(seq_x) == n_steps_in:
          X.append(seq_x)
          y.append(seq_y)
    return np.array(X), np.array(y)

In [54]:
n_steps_in = past_t
X, y = split_sequence(df_rs, n_steps_in)

In [55]:
test_size = 200
train_size = len(X) - test_size
X_train = X[0:train_size]
y_train = y[0:train_size]
X_test = X[train_size:len(X)]
y_test = y[train_size:len(X)]

scaler = MinMaxScaler(feature_range=(-1,1)) #Try this
#scaler = StandardScaler()
X_train = scaler.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)
X_test = scaler.transform(X_test.reshape(-1, X_test.shape[-1])).reshape(X_test.shape)
X_new = scaler.transform(X.reshape(-1, X.shape[-1])).reshape(X.shape)
# save the scaler
dump(scaler, open(folder + 'scaler.pkl', 'wb'))

print("Shape X_train --> " + str(X_train.shape))
print("Shape y_train --> " + str(y_train.shape))
print("Shape X_test  --> " + str(X_test.shape))
print("Shape y_test  --> " + str(y_test.shape))

Shape X_train --> (1234, 7, 91)
Shape y_train --> (1234, 3)
Shape X_test  --> (200, 7, 91)
Shape y_test  --> (200, 3)


# Model

In [56]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers
from tensorflow.keras import activations

In [57]:
# Get square shape
image_shape = X_train.shape[1]

if X_train.shape[1]<X_train.shape[2]:
  image_shape = X_train.shape[2]

# Load Resnet model
base_model = tf.keras.applications.ResNet152V2(
    weights='imagenet',
    input_shape=(image_shape, image_shape, 3),
    include_top=False) 
base_model.trainable = False

# Create input as image
inputs = layers.Input(shape=(X_train.shape[1], X_train.shape[2]))
if X_train.shape[1]<X_train.shape[2]:
  input_img = layers.Reshape((X_train.shape[2], X_train.shape[1]))(inputs) 
  input_img = layers.Dense(image_shape, activation='tanh')(input_img) # Create a square matrix 
  input_img = layers.Reshape((image_shape, image_shape, 1))(input_img) # Reshape into an image with 1 channels
  input_img = layers.Dense(3, activation='tanh', name='image_inp')(input_img) # Learn 3 channels
else:
  input_img = layers.Dense(image_shape, activation='tanh')(inputs) # Create a square matrix 
  input_img = layers.Reshape((image_shape, image_shape, 1))(input_img) # Reshape into an image with 1 channels
  input_img = layers.Dense(3, activation='tanh', name='image_inp')(input_img) # Learn 3 channels
# With tahn all outputs are between -1 and 1 (ResNet needs that)

# Create Model
x = base_model(input_img, training=False)
x = layers.GlobalAveragePooling2D()(x)
#x = layers.Dropout(0.2)(x) 

# Output layer
predictions = layers.Dense(y_train.shape[1], activation="softmax")(x) #Relu for getting positive values
model = tf.keras.Model(inputs, predictions)

model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.01), metrics='accuracy')
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 7, 91)]           0         
                                                                 
 reshape_2 (Reshape)         (None, 91, 7)             0         
                                                                 
 dense_2 (Dense)             (None, 91, 91)            728       
                                                                 
 reshape_3 (Reshape)         (None, 91, 91, 1)         0         
                                                                 
 image_inp (Dense)           (None, 91, 91, 3)         6         
                                                                 
 resnet152v2 (Functional)    (None, 3, 3, 2048)        58331648  
                                                                 
 global_average_pooling2d_1   (None, 2048)             0   

In [58]:
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint

early_stopping = EarlyStopping(monitor='loss', patience=10, verbose=1, mode='auto')
learning_rate_reduction = ReduceLROnPlateau(monitor='loss', patience=7, verbose=1, factor=0.5, min_lr=5e-5)

In [59]:
history = model.fit(X_train, y_train, epochs = 700, batch_size = 16, 
                    callbacks=[learning_rate_reduction, early_stopping],
                    validation_data= (X_test, y_test))

Epoch 1/700
78/78 [==============================] - 20s 142ms/step - loss: 1.1552 - accuracy: 0.3784 - val_loss: 1.1701 - val_accuracy: 0.3200 - lr: 0.0100
Epoch 2/700
78/78 [==============================] - 7s 94ms/step - loss: 1.1086 - accuracy: 0.3793 - val_loss: 1.1152 - val_accuracy: 0.3350 - lr: 0.0100
Epoch 3/700
78/78 [==============================] - 7s 94ms/step - loss: 1.0943 - accuracy: 0.3857 - val_loss: 1.1840 - val_accuracy: 0.3500 - lr: 0.0100
Epoch 4/700
78/78 [==============================] - 7s 94ms/step - loss: 1.1007 - accuracy: 0.4052 - val_loss: 1.1724 - val_accuracy: 0.3700 - lr: 0.0100
Epoch 5/700
78/78 [==============================] - 8s 97ms/step - loss: 1.0948 - accuracy: 0.3906 - val_loss: 1.1176 - val_accuracy: 0.3750 - lr: 0.0100
Epoch 6/700
78/78 [==============================] - 7s 95ms/step - loss: 1.0876 - accuracy: 0.4100 - val_loss: 1.1556 - val_accuracy: 0.3550 - lr: 0.0100
Epoch 7/700
78/78 [==============================] - 8s 96ms/step - 

KeyboardInterrupt: ignored

In [ ]:
plt.figure(figsize=(16,5))
plt.plot(history.history['loss'], label='train', marker='*')
plt.plot(history.history['val_loss'], label='validation', marker='*')
plt.grid()
plt.legend(fontsize=11)

## Fine-tuning

In [ ]:
model_path = folder + 'modelo_best.h5'
checkpoint = ModelCheckpoint(model_path, monitor="val_loss",
                             save_freq = 'epoch', save_best_only=True, mode='min') 

In [22]:
# Unfreeze the base model
base_model.trainable = True

# Very low learning rate 
model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5), metrics='accuracy') 

history = model.fit(X_train, y_train, epochs = 100, batch_size = 16,
                    validation_data= (X_test, y_test),
                    callbacks=[early_stopping, checkpoint])

Epoch 1/100
77/77 [==============================] - 54s 171ms/step - loss: 6.1896 - accuracy: 0.4336 - val_loss: 1.3790 - val_accuracy: 0.3600
Epoch 2/100
77/77 [==============================] - 11s 140ms/step - loss: 1.0004 - accuracy: 0.5123 - val_loss: 1.4159 - val_accuracy: 0.3200
Epoch 3/100
77/77 [==============================] - 11s 139ms/step - loss: 0.9726 - accuracy: 0.5361 - val_loss: 1.4114 - val_accuracy: 0.3850
Epoch 4/100
77/77 [==============================] - 11s 142ms/step - loss: 0.9470 - accuracy: 0.5598 - val_loss: 1.4665 - val_accuracy: 0.3700
Epoch 5/100
77/77 [==============================] - 11s 141ms/step - loss: 0.8901 - accuracy: 0.5754 - val_loss: 1.6790 - val_accuracy: 0.3300
Epoch 6/100
77/77 [==============================] - 14s 185ms/step - loss: 0.9030 - accuracy: 0.5820 - val_loss: 1.5817 - val_accuracy: 0.3450
Epoch 7/100
77/77 [==============================] - 11s 143ms/step - loss: 0.8128 - accuracy: 0.6090 - val_loss: 2.1243 - val_accuracy:

KeyboardInterrupt: ignored

In [ ]:
plt.figure(figsize=(16,5))
plt.plot(history.history['loss'], label='train', marker='*')
plt.plot(history.history['val_loss'], label='validation', marker='*')
plt.grid()
plt.legend(fontsize=11)

In [ ]:
best_model = tf.keras.models.load_model(model_path)
best_model.evaluate(X_test, y_test)

# Results

In [ ]:
y_pred_test = best_model.predict(X_test, verbose=1)
y_pred_redes = best_model.predict(X_new, verbose=1)

## Confusion matrix